

### 1. app.py

```python
from flask import Flask, render_template, request, jsonify
import pickle
import numpy as np

app = Flask(__name__)
```
This imports the necessary libraries:
- `Flask` for creating the web application
- `render_template` to render HTML files
- `request` to handle HTTP requests
- `jsonify` to return JSON responses
- `pickle` to load the saved model
- `numpy` for array operations

```python
def load_model():
    try:
        with open('model.pkl', 'rb') as file:
            model = pickle.load(file)
        return model
    except Exception as e:
        print(f"Error loading model: {e}")
        return None
```
This function loads your saved machine learning model from the model.pkl file, handling any potential errors during loading.

```python
@app.route('/')
def home():
    return render_template('index.html')
```
This defines the root route ("/") which renders the index.html template when someone visits your website.

```python
@app.route('/predict', methods=['POST'])
def predict():
    try:
        # Get the data from the form
        cgpa = float(request.form['cgpa'])
        iq = float(request.form['iq'])
        
        # Load the model and scaler
        model = load_model()
        if model is None:
            return jsonify({'error': 'Model not loaded properly'})
        
        # Load the scaler
        with open('scaler.pkl', 'rb') as file:
            scaler = pickle.load(file)
        
        # Scale the input data
        input_data = np.array([[cgpa, iq]])
        input_data_scaled = scaler.transform(input_data)
        
        # Make prediction
        prediction = model.predict(input_data_scaled)[0]
        
        # Get prediction probability
        prob = model.predict_proba(input_data_scaled)[0][1]
        
        # Return the prediction and probability
        result = {
            'prediction': int(prediction),
            'probability': round(float(prob) * 100, 2),
            'message': 'Placement Likely' if prediction == 1 else 'Placement Unlikely'
        }
        
        return jsonify(result)
    except Exception as e:
        return jsonify({'error': str(e)})
```
This defines the `/predict` route which:
1. Extracts CGPA and IQ values from the form submission
2. Loads your trained model
3. Loads your StandardScaler
4. Scales the input values using the same scaler you used during training
5. Makes a prediction using your model
6. Calculates the confidence probability
7. Returns the prediction result as JSON with:
   - The binary prediction (0 or 1)
   - The confidence probability as a percentage
   - A human-readable message

```python
if __name__ == '__main__':
    app.run(debug=True)
```
This runs the Flask application in debug mode when the script is executed directly.

### 2. templates/index.html

The HTML file creates the interface for your web application. It includes:

- A responsive layout with nice styling
- Input fields for IQ and CGPA with increment/decrement buttons
- A submit button to trigger the prediction
- A result area that will display the prediction

Key parts:
- `<form id="prediction-form">` creates the form that collects user inputs
- The input fields have event handlers for the increment/decrement buttons
- Links to the CSS and JavaScript files using Flask's `url_for()` function

### 3. static/css/style.css

The CSS file handles the styling of your application, making it visually appealing and user-friendly. It includes:

- Responsive layout using modern CSS
- Consistent color scheme
- Well-styled form elements
- Button hover effects
- Result display styling

### 4. static/js/script.js

The JavaScript file handles the interactive elements:

```javascript
function incrementValue(id) {
    // Increases the value of an input field
}

function decrementValue(id) {
    // Decreases the value of an input field
}
```
These functions handle the increment/decrement buttons, ensuring values stay within allowed ranges and displaying the correct number of decimal places for CGPA.

```javascript
document.addEventListener('DOMContentLoaded', function() {
    document.getElementById('prediction-form').addEventListener('submit', function(e) {
        e.preventDefault();
        
        const formData = new FormData(this);
        
        fetch('/predict', {
            method: 'POST',
            body: formData
        })
        .then(response => response.json())
        .then(data => {
            // Display the prediction result
        })
        .catch(error => {
            // Handle errors
        });
    });
});
```
This code:
1. Adds an event listener to the form submission
2. Prevents the default form submission (which would reload the page)
3. Creates FormData from the form inputs
4. Sends an AJAX request to your `/predict` endpoint
5. Processes the response and updates the UI with the prediction result
6. Handles any errors that occur during the process

The key feature is using `fetch()` to send the form data to your Flask backend without refreshing the page, creating a smooth user experience.

### How Everything Works Together

1. The user visits your website and sees the form with input fields for IQ and CGPA
2. They can use the +/- buttons to adjust values or type them directly
3. When they click "Predict", JavaScript sends the data to your Flask backend
4. The Flask application:
   - Processes the input data
   - Uses your StandardScaler to scale the values
   - Feeds the scaled values to your trained model
   - Gets the prediction and probability
   - Returns the results as JSON
5. The JavaScript receives this data and updates the page to show the prediction result

This creates a complete end-to-end machine learning application that lets users interact with your trained model through a user-friendly web interface.